<a href="https://colab.research.google.com/github/burgauss/RL_ControlPurposes/blob/main/MountainCar_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install gym

In [2]:
import numpy as np

In [5]:
#Description of the MountainCarEnv
'''
    Description:
        The agent (a car) is started at the bottom of a valley. For any given
        state the agent may choose to accelerate to the left, right or cease
        any acceleration.
    Source:
        The environment appeared first in Andrew Moore's PhD Thesis (1990).
    Observation:
        Type: Box(2)
        Num    Observation               Min            Max
        0      Car Position              -1.2           0.6
        1      Car Velocity              -0.07          0.07
    Actions:
        Type: Discrete(3)
        Num    Action
        0      Accelerate to the Left
        1      Don't accelerate
        2      Accelerate to the Right
        Note: This does not affect the amount of velocity affected by the
        gravitational pull acting on the car.
    Reward:
         Reward of 0 is awarded if the agent reached the flag (position = 0.5)
         on top of the mountain.
         Reward of -1 is awarded if the position of the agent is less than 0.5.
    Starting State:
         The position of the car is assigned a uniform random value in
         [-0.6 , -0.4].
         The starting velocity of the car is always assigned to 0.
    Episode Termination:
         The car position is more than 0.5
         Episode length is greater than 200
'''

"\n    Description:\n        The agent (a car) is started at the bottom of a valley. For any given\n        state the agent may choose to accelerate to the left, right or cease\n        any acceleration.\n    Source:\n        The environment appeared first in Andrew Moore's PhD Thesis (1990).\n    Observation:\n        Type: Box(2)\n        Num    Observation               Min            Max\n        0      Car Position              -1.2           0.6\n        1      Car Velocity              -0.07          0.07\n    Actions:\n        Type: Discrete(3)\n        Num    Action\n        0      Accelerate to the Left\n        1      Don't accelerate\n        2      Accelerate to the Right\n        Note: This does not affect the amount of velocity affected by the\n        gravitational pull acting on the car.\n    Reward:\n         Reward of 0 is awarded if the agent reached the flag (position = 0.5)\n         on top of the mountain.\n         Reward of -1 is awarded if the position of the 

In [84]:
import gym

env = gym.make("MountainCar-v0")
print("the observation space, upper limits are ", env.observation_space.high) #upper part
print("the observation space, lower limits are ", env.observation_space.low)  # lower part
print("the action space is", env.action_space.n)
env.reset()

the observation space, upper limits are  [0.6  0.07]
the observation space, lower limits are  [-1.2  -0.07]
the action space is 3


array([-0.53076316,  0.        ])

In [105]:
#Hyperparameters
LEARNING_RATE = 0.1
DISCOUNT = 0.95
EPISODES = 20000
SIZE_DISCRETE = 20
SHOW_EVERY = 1000

In [106]:
#Creation of Q_table

DISCRETE_OS_SIZE = [SIZE_DISCRETE] * len(env.observation_space.high) # Matrix to help us define
discrete_os_win_size = (env.observation_space.high - env.observation_space.low) / DISCRETE_OS_SIZE

# Size of q_table is 20x20x3, concatenation of lists
q_table = np.random.uniform(low = -2, high = 0, size = ( DISCRETE_OS_SIZE + [env.action_space.n]))

In [107]:
# Get Discrete Function
# Discrete state will return a tupple with the index of the combination of states
# Each combination of state corresponds to a possibility of three actions according
# to the environment
def get_discrete_state(state):
  discrete_state = (state - env.observation_space.low) / discrete_os_win_size
  return tuple(discrete_state.astype(np.int))

In [ ]:

# When exporting data uncomment following lines
#observation_matrix = []
#reward_vector = []
#done_vector = []
for episode in range(EPISODES):
  discrete_state = get_discrete_state(env.reset())
  done = False
  if episode % SHOW_EVERY == 0:
    print(episode)
  while not done:
    action = np.argmax(q_table[discrete_state])
    new_state, reward, done, _ = env.step(action)
    
    new_discrete_state = get_discrete_state(new_state)

    print(reward, new_state, episode, action)
    
    if not done:
      max_future_q = np.max(q_table[new_discrete_state])
      current_q = q_table[discrete_state + (action, )]
      new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)
      q_table[discrete_state + (action,)] = new_q
    elif new_state[0] >= env.goal_position:
      q_table[discrete_state + (action,)] = 0
      print(f"We made it on episode {episode}")
      #print(new_state)
    
    discrete_state = new_discrete_state

    # For exporting unblock following lines
    #observation_matrix.append(new_state)
    #reward_vector.append(reward)
    #done_vector.append(done)
  


#when exporting ublock following lines
#export_steps(observation_matrix, reward_vector, done_vector)


env.close()


0.5

In [62]:
# function for exporting data

from numpy import asarray
from numpy import savetxt
from google.colab import files

def export_steps(observation_matrix, reward_vec, done_vec):
    
    observation_matrix = np.array(observation_matrix)
    reward_vec = np.array(reward_vec).reshape((len(reward_vec),1))
    done_vec = np.array(done_vec).reshape(len(done_vec),1)
    step_matrix = []
    step_matrix = np.concatenate((observation_matrix, reward_vec, done_vec), axis = 1)
    export_array(step_matrix)


def export_array(_list):

  data = asarray(_list)
  savetxt('data.csv', data, delimiter=',')
  files.download('data.csv')